# Fluxonium-fluxonium system

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pysqkit as sq
from itertools import product, combinations
from matplotlib.lines import Line2D
from IPython.display import display, Latex, Image

In [5]:
# The fluxonia are by default assumed in the double well configuration

fluxonium_a = sq.qubits.Fluxonium(
    label='A',
    joseph_energy=5.8, 
    charge_energy=0.9, 
    induct_energy=0.4
)

fluxonium_b = sq.qubits.Fluxonium(
    label='B',
    joseph_energy=5.8, 
    charge_energy=1.0, 
    induct_energy=0.6
)

fluxonium_a.diagonalize_basis(5)
fluxonium_b.diagonalize_basis(5)

We can get the transition frequencies of fluxonium A and B

In [6]:
qubit_a_tran_freqs = np.diff(fluxonium_a.eig_energies(), 1)
for level in range(3):
    display(Latex(r'$\omega^{{f}}_{{{} \rightarrow {}}}$ = {:.3f} GHz'.format(level, level+1, qubit_a_tran_freqs[level])))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [8]:
qubit_b_tran_freqs = np.diff(fluxonium_b.eig_energies(), 1)
for level in range(3):
    display(Latex(r'$\omega^{{f}}_{{{} \rightarrow {}}}$ = {:.3f} GHz'.format(level, level+1, qubit_b_tran_freqs[level])))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We can now couple fluxonium A and B by introducing a QubitSystem object as

In [10]:
jc = 0.2 # GHz capacitive coupling
system = fluxonium_a.couple_to(fluxonium_b, coupling=sq.couplers.capacitive_coupling, strength=jc)

In [11]:
comp_states = [''.join(qubit_states) for qubit_states in list(product('012', repeat=2))]
q_labels = [qubit.label for qubit in system]

level_inds = xr.DataArray([system.state_index(state) for state in comp_states], dims=['state'], coords=dict(state=comp_states))
level_inds = level_inds.sortby(level_inds)

eig_energies = system.eig_energies(levels=level_inds.data)

trans_freqs = xr.DataArray(
    eig_energies[:,np.newaxis] - eig_energies, 
    dims=['out_state', 'in_state'], 
    coords=dict(
        out_state=level_inds.state.data, 
        in_state=level_inds.state.data
    )
)

mat_elem_amps = xr.DataArray(
    [system.mat_elements(qubit.charge_op(), levels=level_inds.data) for qubit in system],
    dims=['qubit', 'out_state', 'in_state'], 
    coords=dict( 
        qubit=q_labels,
        out_state=level_inds.state.data, 
        in_state=level_inds.state.data
    )
)

In [12]:
trans_freq_11_12 = float(trans_freqs.sel(in_state='11', out_state='21'))
trans_freq_01_02 = float(trans_freqs.sel(in_state='10', out_state='20'))
freq_split = trans_freq_11_12 - trans_freq_01_02
display(Latex(r'$\Delta$ = {:.3f} GHz'.format(freq_split)))

drive_str_ratio = 0.9
drive_str_b = 0.5
drive_str_a = drive_str_ratio*drive_str_b

rabi_freq_11_21 = float(drive_str_a*mat_elem_amps.sel(qubit='A', in_state='11', out_state='21') + 
                        drive_str_b*mat_elem_amps.sel(qubit='B', in_state='11', out_state='21'))
rabi_freq_10_20 = float(drive_str_a*mat_elem_amps.sel(qubit='A', in_state='10', out_state='20') + 
                        drive_str_b*mat_elem_amps.sel(qubit='B', in_state='10', out_state='20'))

rabi_freq_ratio = rabi_freq_11_21/rabi_freq_10_20
display(Latex(r'$\Omega_{{11-21}}/\Omega_{{10-20}}$ = {:.3f}'.format(rabi_freq_ratio)))

_r2 = rabi_freq_ratio**2
drive_freq_ratio = (_r2 - np.sqrt((_r2-1)**2 + _r2))/(_r2-1)
display(Latex(r'$\frac{{\delta}}{{\Delta}}$ = {:.3f}'.format(drive_freq_ratio)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [13]:
mat_elem_amps.sel(qubit='A', in_state='10', out_state='20')

<xarray.DataArray ()>
array(0.6084488)
Coordinates:
    qubit      <U1 'A'
    out_state  <U2 '20'
    in_state   <U2 '10'

In [14]:
mat_elem_amps.sel(qubit='B', in_state='10', out_state='20')

<xarray.DataArray ()>
array(0.03210662)
Coordinates:
    qubit      <U1 'B'
    out_state  <U2 '20'
    in_state   <U2 '10'